In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random as rnd
import seaborn as sns
from math import exp

We Improve on the energy calculation by only calcualting the difference in energy between two states, rather than calculating the energy for the entire state

In [ ]:
def energy(state, J):
    L = state.shape[0]
    e = 0.0
    for i in range(L):
        for j in range(L):
            for k in range(L):
                s = state[i, j, k]
                e -= J * s * state[(i + 1) % L, j, k]
                e -= J * s * state[i, (j + 1) % L, k]
                e -= J * s * state[i, j, (k + 1) % L]
    return e

def neighbor_sum(state, i, j, k):
    L = state.shape[0]
    return (
        state[(i + 1) % L, j, k] +
        state[(i - 1) % L, j, k] +
        state[i, (j + 1) % L, k] +
        state[i, (j - 1) % L, k] +
        state[i, j, (k + 1) % L] +
        state[i, j, (k - 1) % L]
    )

def local_deltaE(state, i, j, k, J):
    s = state[i, j, k]
    nn = neighbor_sum(state, i, j, k)
    return 2.0 * J * s * nn


In [3]:
def metropolis(e_old, e_new, T):
    if e_new < e_old:
        return True
    else:
        if np.random.random() < exp((e_old - e_new) / T):
            return True
        return False
    
def flip(state, i, j, k):
    state[i, j, k] = -state[i, j, k]

def magnetization(state):
    return np.sum(state) / state.size

We implement Wolff's method here, so we can compare it to Metropolis'. Wolff step calcualtes which molecuels to switch, wolff sweep flips the entire cluster that was returned by Wollf step

In [4]:
def wolff_step(state, T, J):
    """
    Perform ONE Wolff cluster flip, h = 0.
    Returns cluster size.
    """
    L = state.shape[0]
    beta = 1.0 / T
    p_add = 1.0 - exp(-2.0 * beta * J)

    # random seed spin
    i0 = np.random.randint(0, L)
    j0 = np.random.randint(0, L)
    k0 = np.random.randint(0, L)
    spin0 = state[i0, j0, k0]

    visited = np.zeros_like(state, dtype=bool)
    stack = [(i0, j0, k0)]
    visited[i0, j0, k0] = True

    cluster = []

    while stack:
        i, j, k = stack.pop()
        cluster.append((i, j, k))

        neighbours = (
            ((i + 1) % L, j, k),
            ((i - 1) % L, j, k),
            (i, (j + 1) % L, k),
            (i, (j - 1) % L, k),
            (i, j, (k + 1) % L),
            (i, j, (k - 1) % L),
        )

        for ni, nj, nk in neighbours:
            if visited[ni, nj, nk]:
                continue
            if state[ni, nj, nk] != spin0:
                continue
            if np.random.random() < p_add:
                visited[ni, nj, nk] = True
                stack.append((ni, nj, nk))

    # flip entire cluster
    for (i, j, k) in cluster:
        state[i, j, k] = -state[i, j, k]

    return len(cluster)


def wolff_sweep(state, T, J):
    """
    Flip clusters until total flipped spins >= N (one 'effective sweep').
    """
    L = state.shape[0]
    N = L ** 3
    flipped = 0
    while flipped < N:
        flipped += wolff_step(state, T, J)


In [5]:
side = 10
t = 1
J = 1.0  
nstep = 50000
seed = 67  
state = 2 * np.random.randint(2, size=(side, side, side)) - 1
np.random.seed(seed)

Here we create a function to run the dynamics, where we can change which method we want to use (Metroplis vs Wolff)

In [ ]:
def run_dynamics(state, J, T, nstep, algo="metropolis"):
    """
    Run nstep Monte Carlo steps with either Metropolis or Wolff.

    state : 3D array (modified in-place)
    J, T  : coupling, temperature
    nstep : number of MC steps
    algo  : "metropolis" or "wolff"

    Returns
    -------
    e_history, m_history : lists of energy and magnetisation per step
    """
    L = state.shape[0]

    e_old = energy(state, J)

    e_history = []
    m_history = []

    for istep in range(nstep):
        if algo == "metropolis":
            # single-spin Metropolis with local ΔE
            i = np.random.randint(L)
            j = np.random.randint(L)
            k = np.random.randint(L)

            dE = local_deltaE(state, i, j, k, J)
            e_new = e_old + dE

            if metropolis(e_old, e_new, T):
                flip(state, i, j, k)
                e_old = e_new
            # else: reject, do nothing

        elif algo == "wolff":
            wolff_step(state, T, J)
            # recompute full energy after the cluster flip
            e_old = energy(state, J)

        else:
            raise ValueError("algo must be 'metropolis' or 'wolff'")

        e_history.append(e_old)
        m_history.append(magnetization(state))

    return e_history, m_history




In [7]:
#e_history, m_history = run_dynamics(state, J, t, nstep, algo="metropolis")

e_history, m_history = run_dynamics(state, J, t, nstep, algo="wolff")

In [39]:
# discard first 1/3 of samples
cut = int(nstep/3)
e_sel = e_history[cut:]
m_sel = m_history[cut:]

avE = np.mean(e_sel)
avM = np.mean(m_sel)

print("Average energy:", avE)
print("Average magnetization:", avM)


Average energy: -2214.3534282328587
Average magnetization: -0.8369612851935739
